In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-7.3.0-posix-seh-rt_v5-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
from matplotlib import rc
rc('figure', figsize=(15, 5))
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from timeit import default_timer as timer

In [3]:
sales_train = pd.read_csv(r'sales_train_v2.csv')
items = pd.read_csv(r'items.csv')
item_categories = pd.read_csv(r'item_categories.csv')
shops = pd.read_csv(r'shops.csv')
sales_test = pd.read_csv(r'test.csv')

In [4]:
sales_test['ID'] = 34
sales_test.rename({'ID': 'date_block_num'}, axis = 1, inplace = True)
sales_test = sales_test.merge(items.loc[:,['item_category_id', 'item_id']], on = 'item_id', how = 'left')

In [5]:
sales_train = sales_train.merge(items.loc[:,['item_category_id', 'item_id']], on = 'item_id', how = 'left')

Create Sales column

In [6]:
sales_train['sales'] = sales_train['item_cnt_day']*sales_train['item_price']

Aggregating the data w.r.t shop id, item id and date_block_num

In [7]:
grouped_data = sales_train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({
    'item_cnt_day':sum,
    'sales':sum,
    'item_price':'median'
}).rename({'item_cnt_day': 'cnt_shop_item', 'sales': 'sales_shop_item', 'item_price': 'price_shop_item'}, axis = 1).reset_index()

Aggregating the data wrt shop_id and date_block_num

In [8]:
aggdata = sales_train.groupby(['date_block_num', 'shop_id']).agg({
    'item_cnt_day':sum,
    'sales':sum,
    'item_price':'median'
}).rename({'item_cnt_day': 'cnt_shop', 'sales': 'sales_shop', 'item_price': 'price_shop'}, axis = 1)
grouped_data = grouped_data.join(aggdata, on = ['date_block_num', 'shop_id'])

Aggregating the data wrt item_id and date_block_num

In [9]:
aggdata = sales_train.groupby(['date_block_num', 'item_id']).agg({
    'item_cnt_day':sum,
    'sales':sum,
    'item_price':'median'
}).rename({'item_cnt_day': 'cnt_item', 'sales': 'sales_item', 'item_price': 'price_item'}, axis = 1)
grouped_data = grouped_data.join(aggdata, on = ['date_block_num', 'item_id'])

Aggregating the data wrt item_category_id and date_block_num

In [10]:
grouped_data = grouped_data.merge(items[['item_id', 'item_category_id']], on = 'item_id', how = 'left')

In [11]:
aggdata = sales_train.groupby(['date_block_num', 'item_category_id']).agg({
    'item_cnt_day':sum,
    'sales':sum,
    'item_price':'median'
}).rename({'item_cnt_day': 'cnt_itcat', 'sales': 'sales_itcat', 'item_price': 'price_itcat'}, axis = 1)
grouped_data = grouped_data.join(aggdata, on = ['date_block_num', 'item_category_id'])

Appending the test data as well

In [13]:
grouped_data = grouped_data.append(sales_test, sort = False).fillna(0)

Splitting date to day, month and year

In [12]:
date_mat = sales_train.date.str.split('.', expand = True)
date_mat.columns = ['day', 'month', 'year']
date_df = pd.concat([sales_train['date_block_num'], date_mat[['month', 'year']]], axis = 1)
date_df.drop_duplicates(inplace = True)
date_df = date_df.append([{'date_block_num': 34, 'month': 11, 'year': 2015}])

In [16]:
grouped_data = grouped_data.merge(date_df, on = 'date_block_num', how = 'left')

In [17]:
grouped_data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'cnt_shop_item',
       'sales_shop_item', 'price_shop_item', 'cnt_shop', 'sales_shop',
       'price_shop', 'cnt_item', 'sales_item', 'price_item',
       'item_category_id', 'cnt_itcat', 'sales_itcat', 'price_itcat', 'month',
       'year'],
      dtype='object')

In [21]:
del aggdata

Creating Expanding Mean Encodings

In [20]:
enc_feat = ['shop_id', 'item_id', 'item_category_id', 'month', 'year']
target_feat = ['cnt_shop_item', 'cnt_shop', 'cnt_item', 'cnt_itcat']

In [27]:
enc_df = grouped_data[['date_block_num', 'shop_id', 'item_id']].copy()
for i in enc_feat:
    for j in target_feat:
        cum_sum = grouped_data.groupby(i).cumsum()[j] - grouped_data[j]
        cum_count = grouped_data.groupby(i).cumcount()
        enc_df[i+'_enc_'+j] = cum_sum/(cum_count+1)   